## Undersøkelse og forberedelse av data

Valg av forklaringvariabler basert på relevans, etikk og data inspection. Eventuelt egenkomponerte forklaringsvariabler.

Før vi kan begynne med modelleringen er vi nødt til å undersøke dataene grundig. Vi må få en oversikt over:
* Størrelsen
* Fordelingen på fasiten vår
* Mangler vi data?
* Finner vi noe rart?
* Hva er kategorisk og hva er numerisk?
* Hva må mappes og hva må ordnes?
* Ser vi noen sammenhenger som utgjør nye variabler?
* Enkoding

#### Vi starter med å lese inn datasettet til en pandas dataframe

In [1]:
import pandas as pd
df = pd.read_csv('data/fraud_oracle.csv')

#### La oss ta en første kikk på dataene, for få oversikt over hva vi skal jobbe med

In [2]:
df.T

,0,1,2,3,4,5,6,7,8,9,...,15410,15411,15412,15413,15414,15415,15416,15417,15418,15419
Month,Dec,Jan,Oct,Jun,Jan,Oct,Feb,Nov,Dec,Apr,...,Nov,Nov,Nov,Nov,Nov,Nov,Nov,Nov,Dec,Dec
WeekOfMonth,5,3,5,2,5,4,1,1,4,3,...,3,4,4,4,4,4,5,5,1,2
DayOfWeek,Wednesday,Wednesday,Friday,Saturday,Monday,Friday,Saturday,Friday,Saturday,Tuesday,...,Sunday,Tuesday,Tuesday,Thursday,Friday,Friday,Thursday,Thursday,Monday,Wednesday
Make,Honda,Honda,Honda,Toyota,Honda,Honda,Honda,Honda,Honda,Ford,...,Dodge,Honda,Pontiac,Mazda,Chevrolet,Toyota,Pontiac,Toyota,Toyota,Toyota
AccidentArea,Urban,Urban,Urban,Rural,Urban,Urban,Urban,Urban,Urban,Urban,...,Urban,Rural,Urban,Urban,Urban,Urban,Urban,Rural,Urban,Urban
DayOfWeekClaimed,Tuesday,Monday,Thursday,Friday,Tuesday,Wednesday,Monday,Tuesday,Wednesday,Wednesday,...,Tuesday,Wednesday,Wednesday,Monday,Monday,Tuesday,Friday,Friday,Thursday,Thursday
MonthClaimed,Jan,Jan,Nov,Jul,Feb,Nov,Feb,Mar,Dec,Apr,...,Nov,Nov,Nov,Nov,Nov,Nov,Dec,Dec,Dec,Dec
WeekOfMonthClaimed,1,4,2,1,2,1,3,4,5,3,...,4,5,4,4,4,5,1,1,2,3
Sex,Female,Male,Male,Male,Female,Male,Male,Male,Male,Male,...,Female,Male,Female,Male,Male,Male,Male,Male,Female,Male
MaritalStatus,Single,Single,Married,Married,Single,Single,Married,Single,Single,Married,...,Single,Married,Single,Married,Single,Married,Married,Single,Married,Single


#### Prosentvis fordeling av fasit

In [3]:
total_fraud=(df.FraudFound_P.value_counts()[1]+df.FraudFound_P.value_counts()[0])
not_fraud=df.FraudFound_P.value_counts()[0]
fraud=df.FraudFound_P.value_counts()[1]
prct_not_fraud=not_fraud/total_fraud*100
prct_fraud=fraud/total_fraud*100
print(f'The data consists of {df.shape[1]} features and {len(df)} observations.')
print(f"{round(prct_not_fraud)}% of the obervations are not fraud")
print(f"{round(prct_fraud)}% of the observations are fraud")

The data consists of 33 features and 15420 observations.
94% of the obervations are not fraud
6% of the observations are fraud


#### Lister alle forklaringsvariabler med korresponderende datatyper 

In [4]:
df.dtypes

Month                   object
WeekOfMonth              int64
DayOfWeek               object
Make                    object
AccidentArea            object
DayOfWeekClaimed        object
MonthClaimed            object
WeekOfMonthClaimed       int64
Sex                     object
MaritalStatus           object
Age                      int64
Fault                   object
PolicyType              object
VehicleCategory         object
VehiclePrice            object
FraudFound_P             int64
PolicyNumber             int64
RepNumber                int64
Deductible               int64
DriverRating             int64
Days_Policy_Accident    object
Days_Policy_Claim       object
PastNumberOfClaims      object
AgeOfVehicle            object
AgeOfPolicyHolder       object
PoliceReportFiled       object
WitnessPresent          object
AgentType               object
NumberOfSuppliments     object
AddressChange_Claim     object
NumberOfCars            object
Year                     int64
BasePoli

#### Lister unike verdier for hver forklaringsvariabel

In [5]:
from pprint import pprint
for col in df:
    print(f"{col}: {df[col].unique()}")       

Month: ['Dec' 'Jan' 'Oct' 'Jun' 'Feb' 'Nov' 'Apr' 'Mar' 'Aug' 'Jul' 'May' 'Sep']
WeekOfMonth: [5 3 2 4 1]
DayOfWeek: ['Wednesday' 'Friday' 'Saturday' 'Monday' 'Tuesday' 'Sunday' 'Thursday']
Make: ['Honda' 'Toyota' 'Ford' 'Mazda' 'Chevrolet' 'Pontiac' 'Accura' 'Dodge'
 'Mercury' 'Jaguar' 'Nisson' 'VW' 'Saab' 'Saturn' 'Porche' 'BMW' 'Mecedes'
 'Ferrari' 'Lexus']
AccidentArea: ['Urban' 'Rural']
DayOfWeekClaimed: ['Tuesday' 'Monday' 'Thursday' 'Friday' 'Wednesday' 'Saturday' 'Sunday'
 '0']
MonthClaimed: ['Jan' 'Nov' 'Jul' 'Feb' 'Mar' 'Dec' 'Apr' 'Aug' 'May' 'Jun' 'Sep' 'Oct'
 '0']
WeekOfMonthClaimed: [1 4 2 3 5]
Sex: ['Female' 'Male']
MaritalStatus: ['Single' 'Married' 'Widow' 'Divorced']
Age: [21 34 47 65 27 20 36  0 30 42 71 52 28 61 38 41 32 40 63 31 45 60 39 55
 35 44 72 29 37 59 49 50 26 48 64 33 74 23 25 56 16 68 18 51 22 53 46 43
 57 54 69 67 19 78 77 75 80 58 73 24 76 62 79 70 17 66]
Fault: ['Policy Holder' 'Third Party']
PolicyType: ['Sport - Liability' 'Sport - Collision' 'Sedan 

#### Vi deler forklaringsvariablene inn i kategoriske og numeriske variabler
Ved hjelp av oversikten over kan vi avgjøre hvilke forklaringsvariabler som skal være kategoriske og hvilke som skal være numeriske. Om vi bestemmer at en forklaringsvariabel skal være kategorisk, kan det allikevel være nyttig å ordne eller sortere verdiene. Det kan hjelpe tremodellen vår til å gjøre bedre og mer effektive valg. Det kan også være nyttig å lage en ny variabel basert på en vi allerede har. For eksempel fra måned til sesong.

#### Vi legger til en ny forkalringsvariabel "Season" ved å mappe Month til Season

In [6]:
season_mapping = {
    "Jan":"Winter",
    "Feb":"Winter",
    "Mar":"Spring",
    "Apr":"Spring",
    "May":"Spring",
    "Jun":"Summer",
    "Jul":"Summer",
    "Aug":"Summer",
    "Sep":"Fall",
    "Oct":"Fall",
    "Nov":"Winter",
    "Dec":"Winter"
}
df["Season"] = df["Month"].map(season_mapping)
print(df["Season"].unique())

['Winter' 'Fall' 'Summer' 'Spring']


#### Mapping av den kategoriske variabelen Month fra string til en ordnet tallrekke av typen int

In [7]:
month_mapping = {
    "Jan":1,
    "Feb":2,
    "Mar":3,
    "Apr":4,
    "May":5,
    "Jun":6,
    "Jul":7,
    "Aug":8,
    "Sep":9,
    "Oct":10,
    "Nov":11,
    "Dec":12
}
df=df.replace({"Month": month_mapping})
print(df["Month"].unique())

[12  1 10  6  2 11  4  3  8  7  5  9]


#### Vi mapper den numeriske variabelen AddressChange_Claim til en ordnet tallrekke


In [8]:
ageofvehicle_mapping = {
    "new": 1,
    "2 years": 2,
    "3 years": 3,
    "4 years": 4,
    "5 years": 5,
    "6 years": 6,
    "7 years": 7,
    "more than 7": 8
}
df=df.replace({"AgeOfVehicle": ageofvehicle_mapping})
print(df["AgeOfVehicle"].unique())

[3 6 7 8 5 1 4 2]


#### Her skriver du inn alle forklaringsvariablene som du vurderer å være kategoriske

In [9]:
categorical_features=['Month','Sex']

#### Vi sjekker om det er duplikater

In [10]:
df.duplicated().any()

False

#### Vi sjekker om det er NaNs i datasettet

In [11]:
df.isna().any()

Month                   False
WeekOfMonth             False
DayOfWeek               False
Make                    False
AccidentArea            False
DayOfWeekClaimed        False
MonthClaimed            False
WeekOfMonthClaimed      False
Sex                     False
MaritalStatus           False
Age                     False
Fault                   False
PolicyType              False
VehicleCategory         False
VehiclePrice            False
FraudFound_P            False
PolicyNumber            False
RepNumber               False
Deductible              False
DriverRating            False
Days_Policy_Accident    False
Days_Policy_Claim       False
PastNumberOfClaims      False
AgeOfVehicle            False
AgeOfPolicyHolder       False
PoliceReportFiled       False
WitnessPresent          False
AgentType               False
NumberOfSuppliments     False
AddressChange_Claim     False
NumberOfCars            False
Year                    False
BasePolicy              False
Season    

#### Er det noen 0-verdier der det ikke bør være det?

In [12]:
for col in df:
    if df[col].dtype == int:
        count = (df[col] == 0).sum()
    else:
        count = (df[col] == '0').sum()
    if count > 0:
        print(f'{col} {count}')

DayOfWeekClaimed 1
MonthClaimed 1
Age 320
FraudFound_P 14497


Ja, vi fant ut at Age-variabelen har så mange som 320 0-verdier, her er det noe som ikke stemmer.
Vi må finne ut hva vi skal mappe denne til. Vi kan enten finne en annen verdi/mean-verdi basert på informasjon fra de andre variablene. Eller vi kan beholde den som 0. I en linær modell er nok en mean verdi viktigere å sette inn enn i en tremodell.

In [13]:
df.loc[df["Age"] == 0] = 16

In [14]:
print(df["Age"].unique())

[21 34 47 65 27 20 36 16 30 42 71 52 28 61 38 41 32 40 63 31 45 60 39 55
 35 44 72 29 37 59 49 50 26 48 64 33 74 23 25 56 68 18 51 22 53 46 43 57
 54 69 67 19 78 77 75 80 58 73 24 76 62 79 70 17 66]


In [15]:
from sklearn.preprocessing import LabelEncoder

#### Encoding vil bare fungere dersom du har mappet alle de numeriske forklaringsvariablene riktig. 

In [ ]:
df[categorical_features]=df[categorical_features].apply(LabelEncoder().fit_transform)
df.to_parquet('fraud_oracle_encoded.parquet')